# Pseudo code for KNN classifier

In [33]:
!pip3 install pandas sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [34]:
import json
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [35]:
pro_wr_data_scores = pd.read_csv('../ProData/WR_career_stats_scored.csv')
college_wr_data = pd.read_csv('../CollegeData/all_receiving.csv')

## Iterate through list of matches and map college id to nfl id

In [36]:
with open('./NFL EDA/matches.json', 'r') as match_file:
  match_data = json.load(match_file)

In [37]:
player_id_tuple = [(match['nfl_id'], match['college_id']) for match in match_data]

nfl_id, college_id = list(zip(*player_id_tuple))
player_ids = pd.DataFrame.from_dict({
  'nfl_id': nfl_id,
  'college_id': college_id
}).set_index('nfl_id')

len(player_ids)

3865

## Reset pro data index to college player id

In [54]:
target = pro_wr_data_scores.set_index('player_id').join(player_ids, how='inner')#.set_index('college_id')
print(len(pro_wr_data_scores))
print(len(target))
target.head()

578
481


,season_count,career_score,college_id
AbbrJa00,3,-0.637018,jared-abbrederis-1
AdamDa01,8,1.446112,davante-adams-1
AdamJo01,1,-0.749130,joe-adams-1
AghoNe00,7,0.439694,nelson-agholor-1
AikeSa00,8,-0.516724,sam-aiken-1


## Split Scored Pro data into tiers for model training / testing

In [53]:
pro_wr_data_scores.head()

,player_id,season_count,career_score
0,AbbrJa00,3,-0.637018
1,AdamDa01,8,1.446112
2,AdamJo01,1,-0.749130
3,AghoNe00,7,0.439694
4,AikeSa00,8,-0.516724


In [55]:
TIERS = 5

sorted_scores = target.sort_values(by='career_score').reset_index()
number_of_players = len(sorted_scores)
increment = number_of_players // TIERS
print(increment)
print(number_of_players)


prev_idx = 0
for tier_num, i in enumerate(range(0, number_of_players, increment)):
    print(tier_num)
    increment_end = prev_idx + increment
    if tier_num + 1 == TIERS:
        print('creating last group')
        sorted_scores.loc[prev_idx:, 'tier'] = tier_num
    else:
        print(f'getting players {prev_idx} - {increment_end}')
        sorted_scores.loc[prev_idx:increment_end, 'tier'] = tier_num

    prev_idx = i + increment
  
target = sorted_scores.set_index('college_id').drop(['season_count', 'career_score'], axis=1)

96
481
0
getting players 0 - 96
1
getting players 96 - 192
2
getting players 192 - 288
3
getting players 288 - 384
4
creating last group
5
getting players 480 - 576


In [56]:
top_tier = target[target.tier == 4]
len(top_tier)

96

In [57]:
with open('../ProData/cleaned/draft_data/WR.json', 'r') as infile:
    data = json.load(infile)

In [65]:
top_tier_nfl_players = top_tier['index'].to_list()

In [66]:
top_tier_draft = {player: draft_data for player, draft_data in data.items() if player in top_tier_nfl_players}

In [69]:
round_ = 0
for _, draft_D in top_tier_draft.items():
    round_ += draft_D['draft_pick']
    
round_ / 96

56.052083333333336

## Clean College

In [40]:
players_to_keep = set(target.index.values).intersection(set(college_wr_data.playerId.values))


In [41]:
len(players_to_keep)

455

In [42]:
grouped_data = college_wr_data.groupby('playerId').sum().loc[players_to_keep]
college_feature_data = grouped_data
for feature in grouped_data:
    if feature == 'g':
        college_feature_data['g'] = grouped_data['g']
        continue
        
    college_feature_data[feature] = grouped_data[feature] / grouped_data['g']

/var/folders/ky/3trdlcm94d9dgq_70q3lbmjr0000gn/T/ipykernel_91694/3872761999.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grouped_data = college_wr_data.groupby('playerId').sum().loc[players_to_keep]


In [43]:
target = target.loc[players_to_keep]
target.sort_values(by='tier', ascending=False).head(50)

/var/folders/ky/3trdlcm94d9dgq_70q3lbmjr0000gn/T/ipykernel_91694/2461575306.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  target = target.loc[players_to_keep]


,tier
college_id,
antonio-brown-1,5.0
nate-burleson-1,4.0
laviska-shenault-jr-1,4.0
terry-mclaurin-1,4.0
stefon-diggs-1,4.0
courtland-sutton-1,4.0
darren-waller-1,4.0
julio-jones-1,4.0
brandon-lafell-1,4.0


In [44]:
target[target.isna().any(axis=1)]

,tier
college_id,


## Train Model

### Split train and test data

In [45]:
features = college_feature_data.sort_index()
target = target.sort_index()
print(len(target), len(features))

455 455


In [46]:
# X -> features, y -> label
X = features
y = target['tier']
 
# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

### Train model

In [47]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 6).fit(X_train, y_train)
 
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
print(accuracy)
 
# creating a confusion matrix
knn_predictions = knn.predict(X_test)
cm = confusion_matrix(y_test, knn_predictions)

0.23684210526315788
